In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import utils.tavr_torch as tavr_torch
from utils.tavr_torch import TAVR_3_Frame, TAVR_1_Frame, TAVR_Sequence, tavr_dataloader
from utils.visualization import display_grid, z_stretch, visualize_frame, set_figsize, get_central_slices
from utils.loss_functions import batch_l2_loss
from utils.run_model import train, test, save, load, get_loss_history
from Models.basic_models import average_model, two_layer_basic, post_process

set_figsize(20, 15)

Training directory found, 36 series
Validation directory found, 6 series
Testing directory found, 10 series


In [2]:
USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

# "Pixl", "Slice", or "None"
preproc_type = "pixel"

validation = TAVR_3_Frame("__valid", preproc=preproc_type)
val_loader = tavr_dataloader(validation, batch_size=4, shuffle=True, num_workers=2)
training = TAVR_3_Frame("__train", preproc=preproc_type)
train_loader = tavr_dataloader(training,batch_size=8, shuffle=True, num_workers=2)


ave_model = average_model()
model = two_layer_basic()
post_proc = post_process(kind=preproc_type).to(device=device)
loss_fn = batch_l2_loss()

# CHANGE TO NAME OF JUPYTER NOTEBOOK
model_name = "Model 4 (Basic+PixelNorm) Run 2 [Adam]"

using device: cpu


In [3]:
learning_rate = 3e-3
reg = 1e-7

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
# Run cell to load model
LOAD = False
iteration_num = -1

if LOAD:
    
    load(model_name, iteration_num, model, optimizer)
    loss_history = get_loss_history(model_name)
    model.to(device=device)
else:
    loss_history = None

In [ ]:
train(model, post_proc, optimizer, train_loader, val_loader, loss_fn, device, 
         model_name, loss_history, epochs=3, print_every=30, print_level=4, lr_decay=1)


Iteration 0, loss = 2.6577, corrected loss = 1236.7100
Validation loss 1089.3878 over 81 frames
model saved to model_checkpoints/Model 4 (Basic+PixelNorm) Run 2 [Adam]/Model 4 (Basic+PixelNorm) Run 2 [Adam]-0
conv_a1.weight,   	norm: 3.9195e+00, 	update norm: 4.4091e-02 	Update/norm: 1.1249e-02
conv_a1.bias,   	norm: 3.9129e-01, 	update norm: 8.4852e-03 	Update/norm: 2.1685e-02
conv_b1.weight,   	norm: 3.7989e+00, 	update norm: 4.4091e-02 	Update/norm: 1.1606e-02
conv_b1.bias,   	norm: 3.0376e-01, 	update norm: 8.4852e-03 	Update/norm: 2.7934e-02
final.weight,   	norm: 1.2531e+00, 	update norm: 1.2000e-02 	Update/norm: 9.5766e-03
final.bias,   	norm: 9.3852e-02, 	update norm: 3.0000e-03 	Update/norm: 3.1965e-02

..

In [ ]:
# One last test + visualize results on 1 validation sequence
val_seq = TAVR_Sequence("__valid")
test(model, val_loader, loss_fn, device)
test(ave_model, val_loader, loss_fn, device)

with torch.no_grad():
    model.eval()
    ave_frames = [val_seq[0][0]]
    for i in range(2,9,2):
        ave_frame = model((val_seq[0][i-2][None,:], val_seq[0][i][None,:]))
        ave_frames += [ave_frame[0][0],  val_seq[0][i]]
    ave_frames += [val_seq[0][9]]
    ave_frames_slices = []
    for f in ave_frames:
        ave_frames_slices += get_central_slices(f)
    set_figsize(6,20)
    display_grid(10, 3, ave_frames_slices)

In [ ]:
optimizer.param_groups[0]['lr']

In [ ]:
for p in model.parameters():
    print(p.norm())